In [ ]:
from google.colab import drive


drive.mount("/content/drive")

In [ ]:
# the RGB scale is calibrated so that when a color's three red/green/blue numbers are equal, the color is a shade of gray
import cv2
import numpy as np
from google.colab.patches import cv2_imshow


BLACK = [0, 0, 0]
GREEN = [0, 128, 0]
DEPTH_THRESHOLD = 5

rgb_image = cv2.imread("/content/drive/My Drive/Colab Notebooks/Research/Eggplant Tracking/data1/14A/RGB/1.jpg")
depth_image = cv2.imread("/content/drive/My Drive/Colab Notebooks/Research/Eggplant Tracking/data1/14A/depth/1.png")

mask = depth_image < DEPTH_THRESHOLD
masked_image = rgb_image * mask
masked_image[np.where((masked_image == BLACK).all(axis=2))] = GREEN

cv2_imshow(rgb_image)
cv2_imshow(masked_image)

In [4]:
import os
import glob


for path_to_rgb in glob.glob("/content/drive/My Drive/Colab Notebooks/Research/Eggplant Tracking/data1/14A/RGB/*.jpg"):
  rgb_image = cv2.imread(path_to_rgb)

  basename = os.path.basename(path_to_rgb)
  basename_png = basename.replace("jpg", "png")
  path_to_depth = os.path.join("/content/drive/My Drive/Colab Notebooks/Research/Eggplant Tracking/data1/14A/depth", basename_png)
  depth_image = cv2.imread(path_to_depth)

  mask = depth_image < DEPTH_THRESHOLD
  masked_image = rgb_image * mask
  masked_image[np.where((masked_image == BLACK).all(axis=2))] = GREEN

  path_to_output = os.path.join("/content/drive/My Drive/Colab Notebooks/Research/Eggplant Tracking/data1/14A/processed_RGB", basename)
  cv2.imwrite(path_to_output, masked_image)

In [ ]:
!pip install motpy
!pip install -r https://raw.githubusercontent.com/ultralytics/yolov5/master/requirements.txt
import re
import torch
from motpy import Detection, MultiObjectTracker

In [ ]:
CONFIDENCE_THRESHOLD = 0.75

model = torch.hub.load("ultralytics/yolov5", "custom", path="/content/drive/My Drive/Colab Notebooks/Research/Yield Prediction Using YOLOv5/model/best.pt", trust_repo=True)
tracker = MultiObjectTracker(dt=0.07)
track_id_dict = {}

fourcc = cv2.VideoWriter_fourcc(*"mp4v")
out = cv2.VideoWriter("output.mov", fourcc, 15.0, (1080, 1920))

for path_to_rgb in sorted(glob.glob("/content/drive/My Drive/Colab Notebooks/Research/Eggplant Tracking/data2/14B/*.jpg"), key=lambda x: int(re.findall(r"\d+", x)[-1])):
  basename = os.path.basename(path_to_rgb)
  basename_without_ext_str = os.path.splitext(basename)[0]
  basename_without_ext_int = int(basename_without_ext_str)
  if basename_without_ext_int % 2 != 0:
    continue

  results = model(path_to_rgb)
  results = results.pred[0].cpu().numpy()

  results = results[results[:, 4] > CONFIDENCE_THRESHOLD]
  results = results[:, :4]
  results = results.tolist()

  tracker.step(detections=[Detection(box=box) for box in results])
  tracks = tracker.active_tracks()

  img = cv2.imread(path_to_rgb)

  for track in tracks:
    pt1 = track.box[:2].astype("uint16")
    pt2 = track.box[2:].astype("uint16")
  
    color = None
    if track.id in track_id_dict:
      color = track_id_dict[track.id]
    else:
      color = np.random.choice(range(256), size=3).tolist()
      track_id_dict[track.id] = color
    
    cv2.rectangle(img, pt1, pt2, color, thickness=3)
  
  x, y, w, h = 5, 5, 335, 32
  sub_img = img[y:y+h, x:x+w]
  white_rectangle = np.full(sub_img.shape, 255, dtype="uint8")

  alpha = 0.5
  beta = 1.0 - alpha
  dst = cv2.addWeighted(sub_img, alpha, white_rectangle, beta, 0.0)

  img[y:y+h, x:x+w] = dst
  cv2.putText(img, f"Eggplant count: {len(track_id_dict):3d}", (10, 32), cv2.FONT_HERSHEY_SIMPLEX, 1.0, BLACK, thickness=2)

  out.write(img)

out.release()